In [1]:
import imaplib
import pyzmail
import requests
import email
import outlook
import re
import openai
import config

In [2]:
# Set up OpenAI API credentials
openai.api_key = config.open_ai_api_key
email_username = config.username
email_password = config.password

In [3]:
mailbox = outlook.Outlook()
mailbox.login(email_username, email_password)
mailbox.inbox()

unread_ids = mailbox.unreadIds()
matching_ids = mailbox.getIdswithWord(unread_ids, 'transaction')  # Pass the correct iterable to the method

# Regular expression patterns
amount_pattern = r'\$(\d+\.\d{2})'
#vendor_pattern = r'\*\s([\w\s]+)'
vendor_pattern = r"with\s(.+)"


# Extract transaction amount and vendor name
transaction_info = []

for id in unread_ids:
    mailbox.getEmail(id)
    subject_line = mailbox.mailsubject()
    amount_match = re.search(amount_pattern, subject_line)
    vendor_match = re.search(vendor_pattern, subject_line)
    if amount_match and vendor_match:
        amount = amount_match.group(1)
        vendor = vendor_match.group(1)
        #print(amount)
        #print(vendor)
        transaction_info.append((amount, vendor))

#print(transaction_info)



Hello World
 > Signed in as budget2396@outlook.com [b'LOGIN completed.']


In [4]:
# Use GPT-3.5 to determine the type of vendor
vendor_categories = ['Food & Drink', 'Bills & Utilities', 'Shopping', 'Other']

for amount, vendor in transaction_info:
    gpt_prompt = f"What type of vendor is '{vendor}'? Classify the vendor into one of the following categories: '{vendor_categories}'. Only return the specific category."
    response = openai.Completion.create(
        engine='text-davinci-003',
        prompt=gpt_prompt,
        max_tokens=100,
        n=1,
        stop=None,
        temperature=0.3
    )
    vendor_type = response.choices[0].text.strip()
    
    print(f"Transaction Amount: ${amount}")
    print(f"Vendor Name: {vendor}")
    print(f"Vendor Type: {vendor_type}")
    print()

Transaction Amount: $15.97
Vendor Name: SQ *ZEDS ICE CREAM
Vendor Type: Food & Drink

Transaction Amount: $36.82
Vendor Name: SQ *TFB OPERATIONS L
Vendor Type: Bills & Utilities

Transaction Amount: $40.00
Vendor Name: HCTRA EZ TAG Rebill
Vendor Type: Bills & Utilities

Transaction Amount: $21.65
Vendor Name: SQ *BOULDIN ACRES
Vendor Type: Food & Drink

Transaction Amount: $9.80
Vendor Name: P. TERRY'S STAND #13
Vendor Type: Food & Drink

